<a href="https://colab.research.google.com/github/RobNova1/test-colab/blob/master/colab_clavis_validation_20190510.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLAVIS VALIDATION - PRICE PROMOTIONS

- Using the Clavis - Price Promotions file downloaded from CLAVIS SITE, NOT AUTOMATED SCRIPT
- Report Date: 2019-05-10

# For Reference:
___

 <img src='https://drive.google.com/open?id=1BCHiW_LZwZRVOhfVa91uLEeQDAXoyey7' width='600'/>
___

content/sample_data/california_housing_train.csv

content/prpromo-2019-05-10.png

** Import pandas as pd.**

In [0]:
import pandas as pd

In [0]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Authenticat Google Cloud.

In [22]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
 GCLOUD_PROJECT='cp-gaa-visualization-dev'

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


** Read Price_-_Products-2019-05-10.csv as a dataframe called prpromo.**

In [0]:
#prpromo = pd.read_csv('Price_-_Products-2019-05-10.csv')

path = "/content/drive/My Drive/colab-data/Price_-_Products-2019-05-10.csv"
prpromo = pd.read_csv(path)

** Check the head of the DataFrame. **

** Use the .info() method to find out how many entries there are.**

In [7]:
prpromo.info() # 148654 Entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 42 columns):
report_date                    888 non-null object
trusted_upc                    140 non-null float64
trusted_rpc                    888 non-null object
trusted_mpc                    106 non-null object
trusted_product_description    888 non-null object
region                         888 non-null object
online_store                   888 non-null object
brand                          888 non-null object
manufacturer                   888 non-null object
category                       888 non-null object
Sub-Category                   888 non-null object
dimension2                     888 non-null object
High Level Segmentation        888 non-null object
Sub-Brand                      888 non-null object
MAP                            888 non-null object
PrimePantry                    888 non-null object
Core SKU                       888 non-null object
dimension8                 

In [0]:
#prpromo.groupby('online_store')['net_price'].sum()

In [0]:
df = prpromo[['trusted_rpc','trusted_product_description', 'online_store', 'net_price', 'discount_percent']]

# Test a few products

In [9]:
df[df['trusted_product_description'].str.contains('Febreze')]

,trusted_rpc,trusted_product_description,online_store,net_price,discount_percent
14,B07BWT78SL,Bounce Plus Febreze Sport Odor Defense Fabric ...,Amazon,11.61,14.7


In [10]:
df[df['trusted_product_description'].str.contains('360')].head()

,trusted_rpc,trusted_product_description,online_store,net_price,discount_percent
20,944139,Colgate 360 4 Zone Whole Mouth Clean Toothbrus...,Walgreens,3.99,13.07
21,B006K3NBVQ,"Colgate 360 Adult Full Head Soft Toothbrush, T...",Amazon,2.99,25.06
22,235889,"Colgate 360 Advanced Optic White Toothbrush, Soft",CVS,3.99,38.52
23,381499,Colgate 360 Charcoal Toothbrush Slimmer Tip So...,CVS,3.99,38.52
24,435546,"Colgate 360 Optic White Full Head Toothbrush, ...",Walgreens,3.99,6.99


In [0]:
#prpromo[prpromo.discount_amount > 0].count()

In [0]:
#prpromo['promotion_type'].unique()

In [0]:
#prpromo['promotion_type'].str.match('Uncategorized').count()

 # create concat field, to drop duplicate values from

In [0]:
prpromo['concat']=prpromo['trusted_rpc']+prpromo['online_store']


In [0]:
prpromo.drop_duplicates('concat', inplace=True)

In [0]:
df = prpromo[['concat','trusted_rpc','trusted_product_description', 'online_store', 'net_price', 'discount_percent']]

In [17]:
df.count()

concat                         771
trusted_rpc                    771
trusted_product_description    771
online_store                   771
net_price                      771
discount_percent               771
dtype: int64

In [18]:
df['concat'].nunique()

771

In [19]:
df.sort_values('discount_percent', ascending=False).head(10)

,concat,trusted_rpc,trusted_product_description,online_store,net_price,discount_percent
356,973249Walgreens,973249,Dove Conditioner Coconut & Hydration,Walgreens,0.30,93.74
659,75661335Target,75661335,Oral-B Rose Gold Genius 9600 Rechargeable Elec...,Target,20.00,90.00
569,75661329Target,75661329,Oral-B Black Genius 9600 Rechargeable Electric...,Target,20.00,90.00
566,75662986Target,75662986,Oral-B 9600 Orchid Purple Electric Toothbrush ...,Target,20.00,90.00
661,75661331Target,75661331,Oral-B Sakura Pink Genius 9600 Rechargeable El...,Target,20.00,90.00
509,75663807Target,75663807,Gain flings! Original Laundry Detergents - 96ct,Target,3.00,88.00
546,16534645Target,16534645,Oral B Cross Action Power Refill - 3 count,Target,4.95,85.83
572,75566000Target,75566000,Oral-B Cross Action Black Refill Heads - 3ct,Target,5.00,85.71
599,75661328Target,75661328,Oral-B Pro 1500 Electric Toothbrush,Target,10.00,85.71
500,52971851Target,52971851,Downy Unstopables In-Wash Original Scent Boost...,Target,1.05,84.98


# BQ Validation

Comparing for same day as above


In [0]:
%%bigquery --project cp-gaa-visualization-dev dfbq
WITH
  PrPromoConcat AS (
  SELECT
    CONCAT(trusted_rpc, online_store) AS concat,
    trusted_rpc,
    trusted_product_description,
    online_store,
    net_price,
    discount_percent
  FROM
    `cp-gaa-visualization-dev.clavis.pricing_promotions`
  WHERE
    report_date = "2019-05-10"
  LIMIT
    1000 ),
  PrPromoCTE AS (
  SELECT
    *,
    ROW_NUMBER()OVER(PARTITION BY concat ORDER BY concat) AS RowNumber
  FROM
    PrPromoConcat )
SELECT
  trusted_rpc,
  trusted_product_description,
  online_store,
  net_price,
  discount_percent
FROM
  PrPromoCTE
WHERE
  RowNumber < 2
ORDER BY
  discount_percent DESC
  

In [37]:
dfbq.head(2)

,trusted_rpc,trusted_product_description,online_store,net_price,discount_percent
0,973249,Dove Conditioner Coconut & Hydration,Walgreens,0.3,93.74
1,75661329,Oral-B Black Genius 9600 Rechargeable Electric...,Target,20,90.00


In [0]:
from google.cloud import bigquery



In [0]:
# sample query from:
# https://cloud.google.com/bigquery/public-data/openaq#which_10_locations_have_had_the_worst_air_quality_this_month_as_measured_by_high_pm10
QUERY = """
        WITH
  PrPromoConcat AS (
  SELECT
    CONCAT(trusted_rpc, online_store) AS concat,
    trusted_rpc,
    trusted_product_description,
    online_store,
    net_price,
    discount_percent
  FROM
    `cp-gaa-visualization-dev.clavis.pricing_promotions`
  WHERE
    report_date = "2019-05-10"
  LIMIT
    1000 ),
  PrPromoCTE AS (
  SELECT
    *,
    ROW_NUMBER()OVER(PARTITION BY concat ORDER BY concat) AS RowNumber
  FROM
    PrPromoConcat )
SELECT
  trusted_rpc,
  trusted_product_description,
  online_store,
  net_price,
  discount_percent
FROM
  PrPromoCTE
WHERE
  RowNumber < 2
ORDER BY
  discount_percent DESC
        """

In [68]:
print(GCLOUD_PROJECT)

cp-gaa-visualization-dev


In [69]:
client = bigquery.Client( GCLOUD_PROJECT)
query_job = client.query(QUERY)
df = query_job.to_dataframe()
df.head(3)


,trusted_rpc,trusted_product_description,online_store,net_price,discount_percent
0,973249,Dove Conditioner Coconut & Hydration,Walgreens,0.3,93.74
1,75661329,Oral-B Black Genius 9600 Rechargeable Electric...,Target,20,90.00
2,75661331,Oral-B Sakura Pink Genius 9600 Rechargeable El...,Target,20,90.00


In [0]:
from google.cloud import bigquery
client = bigquery.Client(GCLOUD_PROJECT)

In [71]:
sql = """
WITH
  PrPromoConcat AS (
  SELECT
    CONCAT(trusted_rpc, online_store) AS concat,
    trusted_rpc,
    trusted_product_description,
    online_store,
    net_price,
    discount_percent
  FROM
    `cp-gaa-visualization-dev.clavis.pricing_promotions`
  WHERE
    report_date = "2019-05-10"
  LIMIT
    1000 ),
  PrPromoCTE AS (
  SELECT
    *,
    ROW_NUMBER()OVER(PARTITION BY concat ORDER BY concat) AS RowNumber
  FROM
    PrPromoConcat )
SELECT
  trusted_rpc,
  trusted_product_description,
  online_store,
  net_price,
  discount_percent
FROM
  PrPromoCTE
WHERE
  RowNumber < 2
ORDER BY
  discount_percent DESC
"""
dfbq = client.query(sql).to_dataframe()
dfbq.head()

,trusted_rpc,trusted_product_description,online_store,net_price,discount_percent
0,973249,Dove Conditioner Coconut & Hydration,Walgreens,0.3,93.74
1,75661329,Oral-B Black Genius 9600 Rechargeable Electric...,Target,20,90.00
2,75661331,Oral-B Sakura Pink Genius 9600 Rechargeable El...,Target,20,90.00
3,75661335,Oral-B Rose Gold Genius 9600 Rechargeable Elec...,Target,20,90.00
4,75662986,Oral-B 9600 Orchid Purple Electric Toothbrush ...,Target,20,90.00


In [72]:
dfbq.count()

trusted_rpc                    771
trusted_product_description    771
online_store                   771
net_price                      771
discount_percent               771
dtype: int64